In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
import numpy as np
from keras.utils import to_categorical 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

Chuẩn bị dữ liệu

In [2]:
data = []
labels = []
classes = 43
# Thư mục hiện tại
cpath = os.getcwd()

for i in range(classes):
    path = cpath + "\\Data\\Train\\" + str(i)
    images = os.listdir(path)  
    for a in images:
        img = cv2.imread(path + '\\' + a)
        img = cv2.resize(img,(30,30))
        img = np.array(img)
        data.append(img)
        labels.append(i)
        
data = np.array(data)
labels = np.array(labels)


In [3]:
print(data.shape, labels.shape)

(39209, 30, 30, 3) (39209,)


Chia training set và testing set


In [4]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [5]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [6]:
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

y_train.shape

(31367, 43)

Build CNN model

In [7]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
model.summary()
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 64)          36928     
                                                        

Train model

In [8]:
anc = model.fit(X_train, y_train, batch_size=64, epochs=15, validation_data=(X_val, y_val))

Epoch 1/15
491/491 [==============================] - 43s 86ms/step - loss: 2.3205 - accuracy: 0.3998 - val_loss: 1.1109 - val_accuracy: 0.6996
Epoch 2/15
491/491 [==============================] - 41s 84ms/step - loss: 0.8869 - accuracy: 0.7318 - val_loss: 0.2714 - val_accuracy: 0.9244
Epoch 3/15
491/491 [==============================] - 44s 89ms/step - loss: 0.5071 - accuracy: 0.8456 - val_loss: 0.2197 - val_accuracy: 0.9467
Epoch 4/15
491/491 [==============================] - 62s 127ms/step - loss: 0.3551 - accuracy: 0.8931 - val_loss: 0.1560 - val_accuracy: 0.9588
Epoch 5/15
491/491 [==============================] - 55s 112ms/step - loss: 0.2844 - accuracy: 0.9157 - val_loss: 0.1210 - val_accuracy: 0.9634
Epoch 6/15
491/491 [==============================] - 51s 105ms/step - loss: 0.2499 - accuracy: 0.9244 - val_loss: 0.1145 - val_accuracy: 0.9670
Epoch 7/15
491/491 [==============================] - 52s 106ms/step - loss: 0.2247 - accuracy: 0.9353 - val_loss: 0.0664 - val_accur

In [9]:
model.save('PhanLoaiBien.h5')